In [ ]:
import os,psycopg2,base64;
from subprocess import PIPE,Popen;
import shlex;
from ipywidgets import IntProgress,HTML,VBox;
from IPython.display import display;

datasets = [
    'nhdplus'
   ,'nhdplus_watersheds'
   ,'nhdplus_indexing'
   ,'nhdplus_navigation30'
   ,'nhdplus_delineation'
   ,'nhdplus_toponet'
   ,'waterspg_support'
   ,'waterspg'
]

cs = "dbname=%s user=%s password=%s host=%s port=%s" % (
    'nhdplus'
   ,'postgres'
   ,os.environ['POSTGRES_PASSWORD']
   ,'dz_pg'
   ,5432
);

try:
    conn = psycopg2.connect(cs);
except:
    raise Exception("database connection error");

loading_dock = "/loading_dock/";

for item in datasets:
   if not os.path.exists(loading_dock + item + ".dmp"):
      raise Exception('specified dump file not found in loading dock: ' + item);

def pg_restore(host_name,host_port,database_name,user_name,database_password,dumpfile,threads=1):

   command = 'pg_restore -c -h {0} -p {1} -d {2} -U {3} -j {5} {4}'\
             .format(host_name,host_port,database_name,user_name,dumpfile,threads);

   command = shlex.split(command);

   p = Popen(command,shell=False,stdin=PIPE,stdout=PIPE,stderr=PIPE);

   return p.communicate(bytes('{}\n'.format(database_password),'utf-8'));

cur = conn.cursor();
cur.execute("DROP SCHEMA IF EXISTS " + ','.join(item) + " CASCADE;");
conn.commit();

i = IntProgress(min=0,max=900,value=100);
label = HTML();
box = VBox(children=[label,i]);
display(box);

for item in datasets:
   label.value = "Loading " + item + " schema...";

   pg_restore(
       'dz_pg'
      ,5432
      ,'nhdplus'
      ,'postgres'
      ,os.environ['POSTGRES_PASSWORD']
      ,'/loading_dock/' + item + '.dmp'
      ,4
   );

   i.value += 100;

i.bar_style = 'success';
label.value = 'All database loads complete';

cur.execute("NOTIFY ddl_command_end;");
conn.commit();
cur.close();
conn.close();
